In [2]:
%reload_ext autoreload
%autoreload 2
from susquehanna_model import susquehannaModel

import pandas as pd
import glob
import tqdm
import numpy as np
import pickle
import os

from ema_workbench import RealParameter, ScalarOutcome, SequentialEvaluator, MultiprocessingEvaluator
from ema_workbench import Model
from ema_workbench import Constraint
import time

In [3]:
from ema_workbench.em_framework.optimization import GenerationalBorg, HyperVolume, EpsilonProgress 

In [4]:
from ema_workbench import ema_logging

In [5]:
inputs = 4 # inputs
outputs = 4 # outputs - just the downstream discharge outputs. 
RBFs = inputs+2 # RBFs
n_years = 50

# Initialize the object
susquehanna_model = susquehannaModel(108.5, 505.0, 5, n_years)

susquehanna_model.load_data()

susquehanna_model.setRBF(RBFs, inputs, outputs)

In [16]:
baseline = {'hydropower_revenue':39.30, 'atomic_power_plant_discharge':0.633,
       'baltimore_discharge':1.00, 'chester_discharge':1.00, 'recreation':0.00, 'environment':0.1058,
       'flood_duration':1.633}

In [6]:
%%time
model = Model("SusquehannaModel", function = susquehanna_model.evaluateMC)

model.outcomes = [ 
                     ScalarOutcome('hydropower_revenue' , kind = ScalarOutcome.MINIMIZE, expected_range=(36.30,100))
                    , ScalarOutcome('atomic_power_plant_discharge', kind = ScalarOutcome.MINIMIZE, expected_range=(0,1))
                    , ScalarOutcome('baltimore_discharge', kind = ScalarOutcome.MINIMIZE, expected_range=(0,1)) 
                    , ScalarOutcome('chester_discharge', kind = ScalarOutcome.MINIMIZE , expected_range=(0,1))
                    , ScalarOutcome('recreation', kind = ScalarOutcome.MINIMIZE , expected_range=(0,1))
                    , ScalarOutcome('environment', kind = ScalarOutcome.MINIMIZE , expected_range=(-1,0))
                    , ScalarOutcome('flood_risk', kind = ScalarOutcome.MINIMIZE , expected_range=(0,100))
                    , ScalarOutcome('flood_duration', kind = ScalarOutcome.MINIMIZE , expected_range=(0,100))]

convergence_metrics = [HyperVolume.from_outcomes(model.outcomes),
                       EpsilonProgress()]

constraints = [Constraint("flood_risk", outcome_names = "flood_risk", function = lambda x:max(0, x-8.8)), Constraint("atomic_power_plant_discharge", outcome_names="atomic_power_plant_discharge",
                          function=lambda x:max(0, 0.9 - (x)))] 
                          


centres = [RealParameter(f"c{i}", -1,1) for i in range(RBFs*(RBFs-inputs))]
radii = [RealParameter(f"r{i}", 0,1) for i in range(RBFs*(RBFs-inputs))]
weights = [RealParameter(f"w{i}", 0,1) for i in range(RBFs*inputs)]
phaseshift = [RealParameter(f"ps{i}", 0,2*np.pi) for i in range(2)]

model.levers = centres + radii + weights + phaseshift

Wall time: 0 ns


In [ ]:
ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(model) as evaluator:
    optimization_results, convergence = evaluator.optimize(searchover='levers', 
                        nfe=1000, epsilons= [0.5, 0.05, 0.05, 0.05, 0.05, 0.001, 0.001, 0.001],convergence=convergence_metrics)

In [8]:
convergence

,hypervolume,epsilon_progress,nfe
0,0.0,0,0


In [18]:
optimization_results

,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,...,w22,w23,ps0,ps1,hydropower_revenue,atomic_power_plant_discharge,baltimore_discharge,chester_discharge,recreation,environment
0,-0.053457,0.847993,0.335180,0.110336,0.893912,0.639466,-0.083350,0.117263,0.987255,0.760006,...,0.253881,0.372884,2.882713,5.862664,-10.746287,-0.006427,-0.004240,-0.003594,377.0,0.102988
1,-0.324739,-0.794333,0.264531,-0.953288,0.109107,0.961874,-0.787242,-0.433681,0.499830,-0.934476,...,0.042587,0.908942,0.007295,2.575407,40.006895,-0.000201,-0.019926,-0.000054,4465.0,0.000764
2,0.308683,0.385227,-0.072128,-0.844474,-0.039459,-0.653750,-0.351243,-0.887383,-0.746584,-0.549271,...,0.607020,0.055860,3.075406,2.540182,-9.081839,-0.006779,-0.010966,-0.003166,2435.0,0.001359
3,-0.507017,0.456497,0.262471,0.666863,0.119483,-0.165630,0.903986,-0.955279,0.323425,0.740255,...,0.710946,0.060452,0.454984,1.523692,32.168593,-0.001395,-0.026530,-0.000626,4857.0,0.000980
4,0.513276,0.971775,-0.961385,-0.373856,-0.581372,-0.420460,-0.561679,0.396369,-0.269306,-0.285217,...,0.200629,0.314731,4.264605,3.615573,-7.402455,-0.004549,-0.002649,-0.007078,349.0,0.332360
5,-0.209843,0.116017,-0.581307,-0.447725,-0.129689,-0.859321,-0.530784,-0.174981,0.401541,0.388643,...,0.107139,0.934192,3.997224,1.071305,40.917608,-0.003522,-0.056511,-0.000834,4759.0,0.014570
6,-0.264501,-0.911997,-0.365421,0.828978,0.853743,-0.947103,0.178522,0.759675,-0.519747,-0.925746,...,0.138746,0.447101,0.898493,3.679340,-20.071442,-0.005967,-0.010316,-0.015385,391.0,0.038102
7,-0.856957,0.361989,-0.583258,-0.805682,0.865211,-0.090239,-0.748457,0.642462,-0.631570,0.937266,...,0.728022,0.092998,2.569469,6.089921,-30.326318,-0.000697,-0.008304,-0.000089,391.0,0.043242
8,-0.711648,0.175000,-0.747284,0.994026,-0.657985,-0.333238,0.570633,-0.811496,0.918356,-0.326813,...,0.422875,0.194457,4.861924,1.773962,-7.760621,-0.005307,-0.028533,-0.000313,1693.0,0.005233
9,-0.644177,0.087937,-0.249821,0.532812,-0.885508,-0.218371,-0.367133,-0.312474,-0.103232,0.876831,...,0.713800,0.449753,2.897332,2.375239,-5.455851,-0.001214,-0.000505,-0.000467,335.0,0.473105
